In [1]:
import nest_asyncio # to allow HABSlib to run async in a Jupyter notebook
nest_asyncio.apply()

import HABSlib as hb

In [2]:
hb.handshake(base_url="http://135.237.144.125", user_id='8d60e8693a9560ee57e8eba3')


       HUMAN        AUGMENTED        BRAIN         SYSTEMS     
   ----------------------------------------------------------- 
   ▒▒▒▒     ▒▒▒▒     ░▒▒▒▒▒░     ▒▒▒▒▒▒▒▒▒▒▒▒░   ░▒▒▒▒▒▒▒▒▒░   
   ▒▒▒▒     ▒▒▒▒    ░▒▒▒▒▒▒▒░             ░▒▒▒▒ ░▒▒▒░     ░▒░  
   ▒▒▒▒▒▒▒▒▒▒▒▒▒   ░▒▒▒▒ ▒▒▒▒░   ▒▒▒▒▒▒▒▒▒▒▒▒▒   ░▒▒▒▒▒▒▒▒▒░   
   ▒▒▒▒     ▒▒▒▒  ░▒▒▒▒   ▒▒▒▒░  ▒▒▒▒     ░▒▒▒▒ ░▒░     ░▒▒▒░  
   ▒▒▒▒     ▒▒▒▒ ░▒▒▒▒     ▒▒▒▒░ ▒▒▒▒▒▒▒▒▒▒▒▒░   ░▒▒▒▒▒▒▒▒▒░   
   ----------------------------------------------------------- 
   version: 0.1.33

Handshake (RSA) successful.
Handshake (AES) successful.


True

In [3]:
from datetime import datetime
from scipy import signal

session_id = hb.send_file(
    user_id='8d60e8693a9560ee57e8eba3', 
    date=datetime.today().strftime('%Y-%m-%d'), 
    edf_file=r"test_data.edf", 
    ch_nrs=[0,1,2,3], 
    session_type="Upload data", 
    tags=['edf', 'file']
)

print("this session:", session_id)

A general error occurred: test_data.edf: can not open file, no such file or directory
this session: False


In [ ]:
import numpy as np

results = hb.get_data_by_session(user_id='8d60e8693a9560ee57e8eba3', session_id='669a6385cc485c74bb95116d')
data = np.array(results)
print(data.shape)
print(data)

In [ ]:
import matplotlib.pyplot as plt

samples_per_second = 250
num_channels=4 # how many you want to plot
    
time = np.arange(data.shape[1]) / samples_per_second
fig, axes = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

if num_channels == 1:
    axes = [axes]

for i, ax in enumerate(axes[:num_channels]):
    ax.plot(time, data[i])
    ax.set_title(f'Channel {i+1}')
    ax.set_ylabel('Amplitude')
    ax.grid(True)

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()

In [ ]:
from scipy import signal

# Define frequency bands
bands = {
    'Delta': (1, 4, 'yellow'),
    'Theta': (4, 8, 'green'),
    'Alpha': (8, 13, 'cyan'),
    'Beta': (13, 30, 'blue'),
    'Gamma': (30, 100, 'red')
}

fig, axes = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

if num_channels == 1:
    axes = [axes]

for i in range(num_channels):
    # Plotting the PSD
    freqs, psd = signal.welch(data[i], fs=samples_per_second)
    axes[i].semilogy(freqs, psd)
    axes[i].set_title(f'Channel {i+1} PSD')
    axes[i].set_ylabel('PSD (V^2/Hz)')
    axes[i].grid(True)

    # Highlighting frequency bands
    for band, (low, high,col) in bands.items():
        axes[i].axvspan(low, high, color=col, alpha=0.3)
        axes[i].text((low + high) / 2, max(psd), band, horizontalalignment='center', verticalalignment='top')

axes[-1].set_xlabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

In [2]:
from scipy import signal
from datetime import datetime

# parameters for the filtering
b_notch, a_notch = signal.iirnotch(50., 2.0, 250)
sos = signal.butter(10, [1, 40], 'bandpass', fs=250, output='sos')

new_session_id, processed_data = hb.process_session_pipe(
    ## Tests ##
    # pipeline='/filtering/artifact/mean',
    # pipeline='/filtering/artifact/std',
    # pipeline='/filtering/artifact/var',
    # pipeline='/filtering/artifact/kurtosis',
    # pipeline='/filtering/artifact/skew',
    # pipeline='/filtering/artifact/ifms',
    # pipeline='/filtering/artifact/delta',
    # pipeline='/filtering/artifact/theta',
    pipeline='/filtering/artifact/alpha',
    # pipeline='/filtering/artifact/beta',
    # pipeline='/filtering/artifact/gamma',
    # pipeline='/filtering/artifact/zerocrossing',
    # pipeline='/filtering/artifact/hjorthmobility',
    # pipeline='/filtering/artifact/hjorthcomplexity',
    # pipeline='/filtering/artifact/entropy',
    # pipeline='/filtering/artifact/fractaldim',
    # pipeline='/filtering/artifact/hurst',
    # pipeline='/filtering/artifact/correlatedim',
    # pipeline='/filtering/artifact/selfaffinity',
    # pipeline='/filtering/artifact/relative',
    # pipeline='/filtering/artifact/asymmetry',
    # pipeline='/filtering/artifact/correlation',
    # pipeline='/filtering/artifact/phaselocking',
    params={ 
        # dictionary, the order does not matter, they will be called by key
        "filtering": {
            'a_notch': a_notch.tolist(),
            'b_notch': b_notch.tolist(),
            'sos': sos.tolist(),
        },
        "artifact":{},
        # "mean":{},
        # "std":{},
        # "var":{},
        # "kurtosis":{},
        # "skew":{},
        # "ifms":{},
        # "delta":{},
        # "theta":{},
        "alpha":{},
        # "beta":{},
        # "gamma":{},
        # "zerocrossing":{},
        # "hjorthmobility":{},
        # "hjorthcomplexity":{},
        # "entropy":{},
        # "fractaldim":{},
        # "hurst":{},
        # "correlatedim":{},
        # "selfaffinity":{},
        # "relative":{'band': 'alpha'},
        # "asymmetry":{'band':'alpha', 'channelA':0, 'channelB':2},
        # "correlation":{'band':'alpha', 'channelA':0, 'channelB':2},
        # "phaselocking":{'band':'alpha', 'channelA':0, 'channelB':2},
    },
    user_id='8d60e8693a9560ee57e8eba3', 
    existing_session_id='669a5fd4cc485c74bb951154', # <<<<<<<<<<<< put here the session_id above
    date=datetime.today().strftime('%Y-%m-%d %H:%M:%S'),
    session_type="post-processing test", 
    tags=['Developer']
)
print("this session:", new_session_id)
print("processed_data:", processed_data)

{'user_id': '8d60e8693a9560ee57e8eba3', 'session_date': '2024-07-19 16:39:49', 'existing_session_id': '669a5fd4cc485c74bb951154', 'session_type': '[On 669a5fd4cc485c74bb951154]: post-processing test', 'session_tags': ['Developer']}
Metadata validation successful!


TypeError: argument should be a bytes-like object or ASCII string, not 'NoneType'

In [ ]:
fig, axes = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

if num_channels == 1:
    axes = [axes]

for i in range(num_channels):
    # Plotting the pipeline results
    axes[i].plot(processed_data[i])
    axes[i].set_title(f'Channel {i+1} pipeline result')

axes[-1].set_xlabel('Time (s)')
plt.tight_layout()
plt.show()